In [0]:

df_eim_agg = spark.sql(f'''
    -- select date_ym
    --     , log_create_time
    --     , context_name
    --     , message_id
    --     , device_name
    --     , platform_code as X_device_platform
    --     , platform_version as X_device_product
    --     , device_id as mac_addr
    --     , name
    --     , type
    --     , spec
    --     , label
    --     , id
    select date_ym, platform_version as X_device_product, count(distinct device_id) as eim_count_ud, count(device_id) as eim_count_log
    from   aic_data_ods.tlamp_view.view__eim__nl_eim_inputlist
    where  country_code = 'US'
      and  ( -- tv_usage_monthly 조건과 동일하게 
            platform_code like 'C%'
            OR (1=1
                and platform_code not like 'C%'
                and sales_model_code not like 'webos%'
                and sales_model_code not like 'WEBOS%'
                and sales_model_code not like 'KEY%'
            )
           )
      and  ( -- 플랫폼별 데이터 수집 기간 
            (platform_version = 'webOSTV 23' and (date_ym >= '2023-01' and  date_ym <= '2025-12')) or 
            (platform_version = 'webOSTV 24' and (date_ym >= '2024-01' and  date_ym <= '2025-12')) or 
            (platform_version = 'webOSTV 25' and (date_ym >= '2025-01' and  date_ym <= '2025-12')) 
           )
    group by date_ym, platform_version
''')

In [0]:

df_mosu_agg = spark.sql(f'''
    with raw_union as (
        -- normal_log_webos23
        select distinct date_ym, X_device_product, mac_addr
        from   aic_data_ods.tlamp.normal_log_webos23
        where  1=1 
        and  context_name = 'tvpowerd'
        and  message_id = 'NL_POWER_STATE'
        and  X_Device_Country = 'US'
        and  ( -- tv_usage_monthly 조건과 동일하게 
                X_device_platform like 'C%'
                OR (1=1
                    and X_device_platform not like 'C%'
                    and x_device_sales_model not like 'webos%'
                    and x_device_sales_model not like 'WEBOS%'
                    and x_device_sales_model not like 'KEY%'
                )
            )
        and  ( -- 플랫폼별 데이터 수집 기간 
                (X_device_product = 'webOSTV 23' and (date_ym >= '2023-01' and  date_ym <= '2025-12')) or 
                (X_device_product = 'webOSTV 24' and (date_ym >= '2024-01' and  date_ym <= '2025-12')) or 
                (X_device_product = 'webOSTV 25' and (date_ym >= '2025-01' and  date_ym <= '2025-12')) 
            )

        union all 

        -- normal_log_webos24
        select distinct date_ym, X_device_product, mac_addr
        from   aic_data_ods.tlamp.normal_log_webos24
        where  1=1 
        and  context_name = 'tvpowerd'
        and  message_id = 'NL_POWER_STATE'
        and  X_Device_Country = 'US'
        and  ( -- tv_usage_monthly 조건과 동일하게 
                X_device_platform like 'C%'
                OR (1=1
                    and X_device_platform not like 'C%'
                    and x_device_sales_model not like 'webos%'
                    and x_device_sales_model not like 'WEBOS%'
                    and x_device_sales_model not like 'KEY%'
                )
            )
        and  ( -- 플랫폼별 데이터 수집 기간 
                (X_device_product = 'webOSTV 23' and (date_ym >= '2023-01' and  date_ym <= '2025-12')) or 
                (X_device_product = 'webOSTV 24' and (date_ym >= '2024-01' and  date_ym <= '2025-12')) or 
                (X_device_product = 'webOSTV 25' and (date_ym >= '2025-01' and  date_ym <= '2025-12')) 
            )

        union all 
        
        -- normal_log_webos25
        select distinct date_ym, X_device_product, mac_addr
        from   aic_data_ods.tlamp.normal_log_webos25
        where  1=1 
        and  context_name = 'tvpowerd'
        and  message_id = 'NL_POWER_STATE'
        and  X_Device_Country = 'US'
        and  ( -- tv_usage_monthly 조건과 동일하게 
                X_device_platform like 'C%'
                OR (1=1
                    and X_device_platform not like 'C%'
                    and x_device_sales_model not like 'webos%'
                    and x_device_sales_model not like 'WEBOS%'
                    and x_device_sales_model not like 'KEY%'
                )
            )
        and  ( -- 플랫폼별 데이터 수집 기간 
                (X_device_product = 'webOSTV 23' and (date_ym >= '2023-01' and  date_ym <= '2025-12')) or 
                (X_device_product = 'webOSTV 24' and (date_ym >= '2024-01' and  date_ym <= '2025-12')) or 
                (X_device_product = 'webOSTV 25' and (date_ym >= '2025-01' and  date_ym <= '2025-12')) 
            )
    )
    
    select date_ym, X_device_product, count(distinct mac_addr) as total_count_ud
    from   raw_union
    group by date_ym, X_device_product
''')

In [0]:
df_result = df_mosu_agg\
    .join(df_eim_agg, ['date_ym','X_device_product'])\
    .select('date_ym','X_device_product','total_count_ud','eim_count_ud','eim_count_log')
display(df_result)

In [0]:
# # s3 csv
# df_23.drop("mac_addr").coalesce(1).write.format("csv").option("header", "true").option("compression", "bzip2").mode("overwrite").save(
#     "s3://s3-lge-he-inbound-aic-dev/HEDS/1171/raw_bz2/webos23"
# )
# df_24.drop("mac_addr").coalesce(1).write.format("csv").option("header", "true").option("compression", "bzip2").mode("overwrite").save(
#     "s3://s3-lge-he-inbound-aic-dev/HEDS/1171/raw_bz2/webos24"
# )
# df_25.drop("mac_addr").coalesce(1).write.format("csv").option("header", "true").option("compression", "bzip2").mode("overwrite").save(
#     "s3://s3-lge-he-inbound-aic-dev/HEDS/1171/raw_bz2/webos25"
# )